# Advent of Code 2018, Dyalog APL edition

To see a correct render of this notebook, check it out on [nbviewer](https://nbviewer.jupyter.org/github/xpqz/AoCDyalog/blob/master/Advent%20of%20Code%202018%20Dyalog%20APL.ipynb).

Annotated solutions in Dyalog APL.

Note that part of the charm of AoC is that every user (or at least groups of users) gets their own unique data set. Some of the solutions below exploit quirks in my particular data set, and so may conceivably not work for the general case.

### Day 1: Chronal Calibration
https://adventofcode.com/2018/day/1

In [38]:
assert←{⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←0}
lines←{⊃⎕NGET ⍵ 1}
ilines←{⍎¨lines ⍵}
line←{⊃⊃⎕NGET ⍵ 1}
]box on -style=max -trains=tree -fns=on
]rows on

┌→─────────────────────────────────────┐
│Was ON -style=max -trains=tree -fns=on│
└──────────────────────────────────────┘

┌→─────┐
│Was ON│
└──────┘

In [39]:
⎕IO←0
DAY1←ilines'data/2018/01.txt'

In [40]:
⊢Part1←+/DAY1
'Expected 477' assert 477=Part1

477

For part 2 we're looking for the first frequency to recur. If we get to the end, take the last value and use that as the start value for the next iteration. We look for the first duplicate using APL's [unique mask](https://aplwiki.com/wiki/Nub_Sieve) function:
```
⍸<\~≠t
    ≠ --- unique mask
   ~  --- flip bits
 <\   --- turn off all 1s after the first
⍸      --- convert to index
```        

In [32]:
Day1p2←{t←⍬⋄0{t,←1↓+\⍺,⍵⋄1=≢e←t[⍸<\~≠t]:⊃e⋄⍵∇⍨⊃¯1↑t}⍵}

In [33]:
⊢Part2←Day1p2 DAY1
'Expected 390' assert 390=Part2

390

### Day 2: Inventory Management System
https://adventofcode.com/2018/day/2

In [59]:
⊢Part1←×/+/⍉↑2 3∘∊¨(≢⊢)⌸¨DAY2←lines'data/2018/02.txt'
'Expected 8118' assert 8118=Part1

8118

In [83]:
⊢Part2←(=⌿l)/0⌷l←↑DAY2[⍸{1∊⍵}¨↓DAY2∘.{+/≠/⍺,⍪⍵}DAY2]
'Expected jbbenqtlaxhivmwyscjukztdp' assert 'jbbenqtlaxhivmwyscjukztdp'≡Part2

┌→────────────────────────┐
│jbbenqtlaxhivmwyscjukztdp│
└─────────────────────────┘